#### Description: Read and Filter SignalK stream from web socket
    Input(s): ip address/web socket info
    Output(s): print info to console
    @author: slawler
    Comments:  Working version for python v3, need to parse source data

In [ ]:
import json
import websocket
import datetime

ws = "ws://demo.signalk.org/signalk/v1/stream?subscribe=all"
stream = websocket.create_connection(ws)
          
#--------Cycle Through streaming data-------#
 
#while stream.connected == True
for _ in range(30):
    try:
        rcvd = stream.recv()
        jdata= json.loads(rcvd)
        #jlog.write(str(jdata)+'\n')
    except(KeyboardInterrupt, SystemExit):
        print("Terminate Test")
        break
#--------Filter & Write data to logs-------#
    try:
        source = jdata['updates'][0]['source']
        table  = jdata['updates'][0]['values'][0]['path'].split('.')[0]
        variables = jdata['updates'][0]['values']
        
        for i, v in enumerate(variables):                
            keys = v.keys()
            
            for key in keys:
                data = variables[i][key]
                
                if isinstance(data,str):                
                    dtype = ''.join(data.split('.')[1:])                 

                elif isinstance(data, dict):
                    dkeys = data.keys()
                    
                    for k in dkeys:
                        datagroup = ''.join(str(variables[i]['path']).split('.')[1:])
                        dtype = datagroup + k
                        value = variables[i][key][k]                        
                        dtm = str(datetime.datetime.now())
                        log_entry = '{}\t{}\t{}\n'.format(dtm,dtype,str(value))
                        #log.write(log_entry)
                        print("logged: {}: {}".format(table,source))
                        
                else:
                    dtm, value = str(datetime.datetime.now()), data
                    log_entry = '{}\t{}\t{}\n'.format(dtm,dtype,str(value))
                    #log.write(log_entry)
                    print("logged: {}: {}".format(table,source))
                    
    except:
        print('ERROR\t\t', jdata,'\n')
        continue